In [1]:
import pandas as pd
import datetime
#import dask.dataframe as dd
from sklearn import preprocessing
import numpy as np

In [2]:
#FOR DATA WITHOUT TICKS
pd.options.mode.chained_assignment = None # Turn off warnings on creating new Year/Month/Time columns

startTime = '2018-01-03 00:00:00'
endTime = '2018-12-21 23:59:59'
SYMBOL = 'GBPJPY'
def prepareDF_signal(dataframe):
    dataframe['Time']=pd.to_datetime(dataframe['Time'], format='%Y.%m.%d %H:%M')
    timeSeries_Start = pd.to_datetime(startTime)
    timeSeries_End = pd.to_datetime(endTime)
    dataframe1 = dataframe.loc[dataframe['Time'] >= timeSeries_Start]
    dataframe3 = dataframe1.loc[dataframe1['Time'] <= timeSeries_End]
    
    return dataframe3

In [3]:
def five(dataframe):
    dataframe = prepareDF_signal(dataframe)
    dataframe.drop(['5_Climactic Bar Strong','5_Climactic Bar Weak','5_HV Effort To Fall','5_HV Effort To Rise', '5_ND SO', '5_NSP RSO',
                                     '5_Pseudo Upthrust','5_Stopping Volume', '5_Supply Test', '5_Smart Buying', '5_Smart Selling'], axis=1)
    dataframe['Year'] = pd.DatetimeIndex(dataframe['Time']).year
    dataframe['Month'] = pd.DatetimeIndex(dataframe['Time']).month
    dataframe['Day'] = pd.DatetimeIndex(dataframe['Time']).day
    dataframe = dataframe[['Time', 'Year', 'Month', 'Day', 'DayOfWeak', 'Hour', 'Minute', 'Open', 'Close', 'High', 'Low', 'Volume', 'RSI', 'MA',
        '5_Buying Climax','5_Reverse Shake Out', '5_Reverse Upthrust', '5_Selling Climax', '5_Shake Out', '5_Upthrust']]       
    return dataframe

In [4]:
def fifteen(dataframe):
    dataframe = prepareDF_signal(dataframe)
    dataframe_short = dataframe[['Time', '15_Shake Out', '15_Reverse Shake Out', '15_Stopping Volume', '15_Supply Test', '15_Climactic Bar Strong', '15_Climactic Bar Weak','15_ND SO',
       '15_NSP RSO']]
    return dataframe_short

In [5]:
def hour(dataframe):
    dataframe = prepareDF_signal(dataframe)
    dataframe_short = dataframe[['Time', '60_ND SO', '60_NSP RSO','60_Reverse Upthrust','60_Upthrust', '60_Smart Buying', '60_Smart Selling',
                                '60_HV Effort To Fall', '60_HV Effort To Rise']]
    return dataframe_short

In [6]:
def day(dataframe):
    dataframe = prepareDF_signal(dataframe)
    dataframe.rename(columns={'MA': '1440_MA', 'RSI': '1440_RSI'}, inplace=True)
    dataframe_short = dataframe[['Time', '1440_MA', '1440_RSI', '1440_Reverse Upthrust','1440_Upthrust','1440_Smart Buying', '1440_Smart Selling',
                                '1440_Pseudo Upthrust', '1440_Pseudo Reverse Upthrust']]
    return dataframe_short

In [54]:
def mergeDF(lowTF,highTF):
    tf_merge = pd.merge(lowTF, highTF, on='Time', how='outer')
    fill_merge = tf_merge.fillna(method='ffill')
    return fill_merge

LOAD CSV FILES WITH SIGNALS

In [45]:
day_csv = pd.read_csv(f'data/GJ_short3/{SYMBOL}_1440.csv')
day_df = day(day_csv)

In [46]:
oneHour_csv = pd.read_csv(f'data/GJ_short3/{SYMBOL}_60.csv')
oneHour_df = hour(oneHour_csv)

In [47]:
fifteen_csv = pd.read_csv(f'data/GJ_short3/{SYMBOL}_15.csv')
fifteen_df = fifteen(fifteen_csv)

In [48]:
five_csv = pd.read_csv(f'data/GJ_short3/{SYMBOL}_5.csv')
five_df = five(five_csv)

MERGE SIGNAL DATAFRAMES

In [55]:
mergefifteen = mergeDF(five_df,fifteen_df)
mergeHour = mergeDF(mergefifteen,oneHour_df)
#mergeFourHour = mergeDF(mergeHour,fourHour)
mergeDay = mergeDF(mergeHour,day_df)

In [58]:
#gf = day_df.iloc[:303,:]
#gf.shape
mergeDay.iloc[2200:,32:44].head(20)

,60_Smart Buying,60_Smart Selling,60_HV Effort To Fall,60_HV Effort To Rise,1440_MA,1440_RSI,1440_Reverse Upthrust,1440_Upthrust,1440_Smart Buying,1440_Smart Selling,1440_Pseudo Upthrust,1440_Pseudo Reverse Upthrust
2200,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2201,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2202,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2203,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2204,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2205,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2206,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2207,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2208,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0
2209,0.0,0.0,0.0,0.0,152.159214,40.913447,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
mergeDay.columns.values

In [14]:
#USED TO TEST FOR NAN VALUES IF ALL TIMEFRAMES AREN'T ALIGNED 100% BY START/END TIME
testPD = mergeDay
null_columns=testPD.columns[testPD.isnull().any()]

testPD[null_columns].isnull().sum()
print(testPD[testPD.isnull().any(axis=1)][null_columns].head())

Empty DataFrame
Columns: []
Index: []


In [51]:
mergeDay.columns = ['Time', 'Year', 'Month', 'Day', 'DayOfWeak', 'Hour', 'Minute',
       'Open', 'Close', 'High', 'Low', 'Volume', 'RSI', 'MA','1440_MA',
       '1440_RSI',
       '5_Buying Climax', '5_Reverse Shake Out', '5_Reverse Upthrust',
       '5_Selling Climax', '5_Shake Out', '5_Upthrust', '15_Shake Out',
       '15_Reverse Shake Out', '15_Stopping Volume', '15_Supply Test',
       '15_Climactic Bar Strong', '15_Climactic Bar Weak', '15_ND SO',
       '15_NSP RSO', '60_ND SO', '60_NSP RSO', '60_Reverse Upthrust',
       '60_Upthrust', '60_Smart Buying', '60_Smart Selling',
       '60_HV Effort To Fall', '60_HV Effort To Rise',  '1440_Reverse Upthrust', '1440_Upthrust',
       '1440_Smart Buying', '1440_Smart Selling', '1440_Pseudo Upthrust',
       '1440_Pseudo Reverse Upthrust']

In [52]:
signalColumns = ['5_Buying Climax', '5_Reverse Shake Out', '5_Reverse Upthrust',
       '5_Selling Climax', '5_Shake Out', '5_Upthrust', '15_Shake Out',
       '15_Reverse Shake Out', '15_Stopping Volume', '15_Supply Test',
       '15_Climactic Bar Strong', '15_Climactic Bar Weak', '15_ND SO',
       '15_NSP RSO', '60_ND SO', '60_NSP RSO', '60_Reverse Upthrust',
       '60_Upthrust', '60_Smart Buying', '60_Smart Selling',
       '60_HV Effort To Fall', '60_HV Effort To Rise',  '1440_Reverse Upthrust', '1440_Upthrust',
       '1440_Smart Buying', '1440_Smart Selling', '1440_Pseudo Upthrust',
       '1440_Pseudo Reverse Upthrust']

CAST SIGNAL COLUMNS BACK TO INTEGER

In [17]:
#merged = data_merged.copy()
merged = mergeDay.copy()
for column in signalColumns:
    merged = merged.astype({column: int})

In [18]:
data_clean = merged.drop(['Time'], axis=1)
data_clean.shape

(72823, 43)

In [35]:
data_clean['1440_MA'][46464]

0

EXPORT

In [19]:
export_csv = data_clean.to_csv(f'data\\{SYMBOL}_2018_5min_short44.csv', index=False, header=None)

In [20]:
data_clean.head()

,Year,Month,Day,DayOfWeak,Hour,Minute,Open,Close,High,Low,...,60_Smart Buying,60_Smart Selling,60_HV Effort To Fall,60_HV Effort To Rise,1440_Reverse Upthrust,1440_Upthrust,1440_Smart Buying,1440_Smart Selling,1440_Pseudo Upthrust,1440_Pseudo Reverse Upthrust
0,2018,1,3,3,0,0,152.629,152.603,152.652,152.603,...,0,0,151,62,0,0,0,0,0,0
1,2018,1,3,3,0,5,152.603,152.644,152.661,152.603,...,0,0,151,62,0,0,0,0,0,0
2,2018,1,3,3,0,10,152.644,152.653,152.668,152.639,...,0,0,151,62,0,0,0,0,0,0
3,2018,1,3,3,0,15,152.654,152.656,152.662,152.644,...,0,0,151,62,0,0,0,0,0,0
4,2018,1,3,3,0,20,152.656,152.669,152.671,152.647,...,0,0,151,62,0,0,0,0,0,0


SCALER

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scl = scaler.fit(data_clean)
scaler_filename = "scaler\\GBPJPY_scaler_Live192.save"
joblib.dump(scaler, scaler_filename) 


In [ ]:
decay = 0.6
lr= 0.01

In [ ]:
epoch = [1,2,3,4,5,6,7,8,9,10]
for i in epoch:
    lr = lr*decay
    print(lr)

In [ ]:
import pandas as pd
data7 = pd.read_csv('data\GBPJPY_2018_5min_NOtick.csv', header=None)

In [ ]:
data7.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
sc_data = scaler.fit_transform(data7)

In [ ]:
import numpy as np
row1 = np.array(sc_data[11654])

In [ ]:
row2 = row1.astype('float16')

In [43]:
data= np.genfromtxt('data\\WORK_10pip_1hour_GBPJPY_2019_5min_short94.csv', delimiter=',')
data[1]

array([0.00000000e+00, 2.01900000e+03, 5.00000000e+00, 1.00000000e+00,
       3.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.45287000e+02,
       1.45220000e+02, 1.45287000e+02, 1.45111000e+02, 2.10000000e+01,
       5.41428116e+01, 1.45285714e+02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.35000000e+02,
       1.45000000e+02, 5.80000000e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.18400000e+03, 1.44000000e+02, 7.20000000e+01, 0.00000000e+00,
      

In [ ]:
data_clean.iloc[:1,:15]

In [ ]:
y = np.empty((data_clean.shape[0],2))
counter = [0,0,0]
for index, row in data_clean.iterrows():
    if index > data_clean.shape[0]-40:
        break
    y[index,0] = index
    price = row.iloc[7]
    for i in range(20):
        if data_clean.iloc[index+i+1][7] > price + 0.2:
            y[index,1] = 1
            counter[1] += 1
            break
        elif data_clean.iloc[index+i+1][7] < price-0.2:
            y[index,1] = 2
            counter[2] += 1
            break
        else:
            if i == 19:
                y[index,1] = 0
                counter[0] += 1
            
print(f'DONE! Zero: {counter[0]}  Ones: {counter[1]}  Twos: {counter[2]}')

In [ ]:
DONE! Zero: 6086  Ones: 30205  Twos: 31922

In [ ]:
y_adj = y[:-40,:]

In [ ]:
#y_adj.shape
df = pd.DataFrame(y_adj)
df.describe()

In [ ]:
#y = pd.DataFrame(y_adj)
#i = 0
#for index, row in y.iterrows():
#    if row.iloc[1] == 2:
#        for i in range(500):
#            if y.iloc[index+i][1] != 2:
#                row.iloc[1] = y.iloc[index+i][1]
#                i += 1
#                break
#print(f'Done! {i}')

In [ ]:
filter_arr = []

# go through each element in arr
for index, row in df.iterrows():
  # if the element is completely divisble by 2, set the value to True, otherwise False
  if row.iloc[1] == 2:
    filter_arr.append(True)
  else:
    filter_arr.append(False)

newarr = df[filter_arr]
newarr.shape

In [ ]:
16700+25098+26415

In [ ]:
data_adj = data_clean.iloc[:-40,:]
data_adj.shape

In [ ]:
y = pd.DataFrame(y_adj)
data_adj['y'] = y[1]

In [ ]:
data_adj

In [ ]:
export_csv = data_adj.to_csv(f'data\\WORK_20pip_1hour_{SYMBOL}_2018_5min_short44.csv')